**Verify Discharge**

Adrian Wiegman

2023-08-12

-------

## Description

This notebook turns raw data from LiDAR DEM, NHD flowlines, and polygons of cranberry bogs, into a layer of polygons representing the topographic catchments draining into each cranberry bog in southeast Massachusetts. 

Even though the lidar data as been hydro flattened/enforced, there are still a number of flowpaths that are not detected underneath highways (e.g. interstate I-495, I-195)

The solution is to "burn" in stream flowlines from the national hydrography dataset. This is done by putting a buffer around the stream flowline network, then assigning an arbitary large value to the stream network polygon, then converting to raster and subtracting the stream elevations from the DEM. 

Once D8 flow direction and flow accumulation rasters have been made the primary objectives can be completed. 

Data Sources:

1. Mass GIS Lidar DEM (1ft vertical resolution, ~1m horizontal resolution)
2. USGS National Hydrography Dataset
- Flow lines
3. Cranberry bogs layer

Steps of Processing: 

1. Prepare Lidar
   - 1M Lidar Elevation -> Clip to study area -> 
   - Resample to 10m resolution using aggregate minimum cell value
   - Fill Sinks 

2. Prepare Flowlines
   - combine flowlines into one layer
   - dissolve flowlines
   - buffer flowlines to 3x the resolution of processed lidar. 
       - buffering width of 15m 
   - convert to raster
3. Burn in flow lines
    - assign a value of -100 feet to flow lines
    - add flowlines to existing elevation (subtract 100 feet)
    
4. fill sinks (again)
5. D8 Flow Direction 
6. Flow Accumulation
7. Generate Bog Pour Points
    - find maximum flow accumulation value inside each bog.
    - generate a point at each bog maximum value.
    - identity to get cranberry bog attributes at each point.
8. Delineate Basins for each point
    - with the bog cranberry points loop through each point
        - delineate watershed using the bog pour point and the D8 flow direction
        - save the output to a temp file with the feature ID ('FID') of the cranberry bog. 
    - merge all cranberry bog basins to one polygon layer, containing the FID of the cranberry bog. 

## Setup Environment

In [25]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = "C:/Users/Adrian.Wiegman/Documents/GitHub/Wiegman_USDA_ARS/MEP/_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_run_script_w_propy_bat
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups
   fn_buildWhereClauseFromList

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic

In [72]:
wdr = r"C:\Workspace\Geodata\Verify_Discharge"
print(wdr)

C:\Workspace\Geodata\Verify_Discharge


In [73]:
# Create new file geodatabase to store results\n",
gdb = "Verify_Discharge.gdb"

ap.env.workspace = os.path.join(wdr,gdb)

In [74]:
arcpy.management.XYTableToPoint(
    in_table=r"C:\Users\Adrian.Wiegman\Documents\GitHub\Wiegman_USDA_ARS\MEP\data\df_Q_bogs_streams.csv",
    out_feature_class=r"C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\df_Q_bogs_streams_XYTableToPoint",
    x_field="Long",
    y_field="Lat",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\Workspace\\Geodata\\Verify_Discharge\\Verify_Discharge.gdb\\df_Q_bogs_streams_XYTableToPoint'>

In [75]:
arcpy.analysis.Buffer(
    in_features="df_Q_bogs_streams_XYTableToPoint",
    out_feature_class=r"C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\df_Q_bogs_streams_XYT_Buffer",
    buffer_distance_or_field="150 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="GEODESIC"
)

<Result 'C:\\Workspace\\Geodata\\Verify_Discharge\\Verify_Discharge.gdb\\df_Q_bogs_streams_XYT_Buffer'>

In [76]:
files = ["C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_D8_gwe_bf",
"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_D8_gwe_bf_lt1m",
"C:\Workspace\Geodata\MEP\Default.gdb\LidAg10BF_FlowAcc",
"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_Dinf_gwe_bf",
"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_Dinf_gwe_bf_lt1m",
"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_MDF_gwe_bf"]
import re
names = [re.search("\.gdb\\\(.*)",x)[1] for x in files]

In [77]:
in_feat = "df_Q_bogs_streams_XYT_Buffer"
for i in range(len(files)):
    out_raster = arcpy.ia.ZonalStatistics(
        in_zone_data=in_feat,
        zone_field="OBJECTID",
        in_value_raster=files[i],
        statistics_type="MAXIMUM",
        ignore_nodata="DATA",
        process_as_multidimensional="CURRENT_SLICE",
        percentile_value=90,
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360)
    out_rast_name = "ZS_MAX_verify_"+names[i]
    out_raster.save(out_rast_name)
    

In [78]:
# generate string of rasters to 
long_names = ["ZS_MAX_verify_"+n for n in names]
long_names
_ = ["{} {}".format(os.path.join(wdr,gdb,n),n) for n in long_names]
in_rasters = ";".join(_)
print(in_rasters)

C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\ZS_MAX_verify_FA_D8_gwe_bf ZS_MAX_verify_FA_D8_gwe_bf;C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\ZS_MAX_verify_FA_D8_gwe_bf_lt1m ZS_MAX_verify_FA_D8_gwe_bf_lt1m;C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\ZS_MAX_verify_LidAg10BF_FlowAcc ZS_MAX_verify_LidAg10BF_FlowAcc;C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\ZS_MAX_verify_FA_Dinf_gwe_bf ZS_MAX_verify_FA_Dinf_gwe_bf;C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\ZS_MAX_verify_FA_Dinf_gwe_bf_lt1m ZS_MAX_verify_FA_Dinf_gwe_bf_lt1m;C:\Workspace\Geodata\Verify_Discharge\Verify_Discharge.gdb\ZS_MAX_verify_FA_MDF_gwe_bf ZS_MAX_verify_FA_MDF_gwe_bf


In [79]:
arcpy.sa.ExtractMultiValuesToPoints(
    in_point_features="df_Q_bogs_streams_XYTableToPoint",
    in_rasters=in_rasters,
    bilinear_interpolate_values="NONE")

<geoprocessing server result object object at 0x000001FE297E3630>

In [80]:
# cell size for grid where lt1m logic was applied to bogs. 
# Compute conversion factor to translate 
cell_size = 13.778565541850835
FA_to_Q = (cell_size)**2 * 27.25 * 2.54 * (1/100) * (1/365.25)
FA_to_Q

0.35976425532343725

In [83]:
# Compute conversion factor to translate 
cell_size = 10
FA_to_Q = (cell_size)**2 * 27.25 * 2.54 * (1/100) * (1/365.25)
FA_to_Q

0.1895003422313484

In [81]:
arcpy.management.CalculateField(
    in_table="df_Q_bogs_streams_XYTableToPoint",
    field="Q_m3d_FA1",
    expression="!{}!*{}".format(out_rast_name,FA_to_Q),
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
TypeError: unsupported operand type(s) f

<Result 'C:\\Workspace\\Geodata\\Verify_Discharge\\Verify_Discharge.gdb\\df_Q_bogs_streams_XYTableToPoint'>

In [82]:
arcpy.conversion.TableToExcel(
    Input_Table="df_Q_bogs_streams_XYTableToPoint",
    Output_Excel_File=r"C:\Workspace\Geodata\Verify_Discharge\outputs\df_Q_bogs_streams_XYTableToPoint_TableToExcel.xls",
    Use_field_alias_as_column_header="NAME",
    Use_domain_and_subtype_description="CODE"
)

<Result 'C:\\Workspace\\Geodata\\Verify_Discharge\\outputs\\df_Q_bogs_streams_XYTableToPoint_TableToExcel.xls'>

In [7]:
files = ["C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_D8_gwe_bf",
"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_D8_gwe_bf_lt1m",
"C:\Workspace\Geodata\MEP\Default.gdb\LidAg10BF_FlowAcc",
"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_Dinf_gwe_bf",
"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\FA_Dinf_gwe_bf_lt1m"]
import re
names = [re.search("\.gdb\\\(.*)",x)[1] for x in files]

In [11]:
_ = ["{} {}".format(files[i],names[i]) for i in range(len(files))]
in_rasters = ";".join(_)
# resume here. make string for multiple files raster extract

In [13]:
in_feat = "df_Q_bogs_streams_XYT_Buffer"
for i in range(len(files)):
    out_raster = arcpy.ia.ZonalStatistics(
    in_zone_data=in_feat,
    zone_field="OBJECTID",
    in_value_raster=files[i],
    statistics_type="MAXIMUM",
    ignore_nodata="DATA",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_value=90,
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360)
    out_rast_name = "ZS_MAX_"+names[i]
    out_raster.save(out_rast_name)

NameError: name 'in_feat' is not defined

FA = Flow accumulation (number of cells draining to a point)
w = width of cell in distance units (10 meters)
l = length of cell in distance units (10 meters)

A = FA * cell_size^2

cell size is the side length of the grid cells

A = FA * 100 m^2/cell

Q = A * r

where r is recharge rate 
27.25 in/yr 

m3/d =  27.25 in/yr * 2.54 cm/in * 1/100 m/cm *  1/365.25 yr/d

